# Pet Adoption Prediction Project

## Authors
[Your Name Here]

## Abstract
This project aims to predict the likelihood of pet adoption based on various features such as pet type, breed, age, health condition, and other characteristics. We analyze a dataset of shelter pets and build machine learning models to identify key factors that influence adoption success. The insights from this analysis can help animal shelters optimize their adoption strategies and improve outcomes for pets in need of homes.1

## 1. Setup

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Setup complete!")

Setup complete!


## 2. Load Data

In [10]:
# Load the dataset
df = pd.read_csv('pet_adoption_data.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

Dataset loaded successfully!
Shape: (2007, 13)

First few rows:


,PetID,PetType,Breed,AgeMonths,Color,Size,WeightKg,Vaccinated,HealthCondition,TimeInShelterDays,AdoptionFee,PreviousOwner,AdoptionLikelihood
0,500,Bird,Parakeet,131,Orange,Large,5.039768,1,0,27,140,0,0
1,501,Rabbit,Rabbit,73,White,Large,16.086727,0,0,8,235,0,0
2,502,Dog,Golden Retriever,136,Orange,Medium,2.076286,0,0,85,385,0,0
3,503,Bird,Parakeet,97,White,Small,3.339423,0,0,61,217,1,0
4,504,Rabbit,Rabbit,123,Gray,Large,20.498100,0,0,28,14,1,0


In [11]:
# Dataset information
print("Dataset Information:")
print("="*50)
df.info()
print("\n" + "="*50)
print("\nDataset Statistics:")
df.describe()

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2007 entries, 0 to 2006
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PetID               2007 non-null   int64  
 1   PetType             2007 non-null   object 
 2   Breed               2007 non-null   object 
 3   AgeMonths           2007 non-null   int64  
 4   Color               2007 non-null   object 
 5   Size                2007 non-null   object 
 6   WeightKg            2007 non-null   float64
 7   Vaccinated          2007 non-null   int64  
 8   HealthCondition     2007 non-null   int64  
 9   TimeInShelterDays   2007 non-null   int64  
 10  AdoptionFee         2007 non-null   int64  
 11  PreviousOwner       2007 non-null   int64  
 12  AdoptionLikelihood  2007 non-null   int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 204.0+ KB


Dataset Statistics:


,PetID,AgeMonths,WeightKg,Vaccinated,HealthCondition,TimeInShelterDays,AdoptionFee,PreviousOwner,AdoptionLikelihood
count,2007.000000,2007.000000,2007.000000,2007.000000,2007.000000,2007.000000,2007.000000,2007.000000,2007.000000
mean,1503.000000,92.279522,15.705776,0.701046,0.196313,43.974091,249.142003,0.301943,0.328351
std,579.515315,52.148363,8.327749,0.457914,0.397307,25.740253,142.887040,0.459215,0.469730
min,500.000000,1.000000,1.018198,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1001.500000,48.000000,8.730396,0.000000,0.000000,21.000000,127.000000,0.000000,0.000000
50%,1503.000000,94.000000,15.925416,1.000000,0.000000,45.000000,242.000000,0.000000,0.000000
75%,2004.500000,138.000000,22.737180,1.000000,0.000000,66.000000,375.000000,1.000000,1.000000
max,2506.000000,179.000000,29.995628,1.000000,1.000000,89.000000,499.000000,1.000000,1.000000


### Target Variable: AdoptionLikelihood

**AdoptionLikelihood** is our target variable (dependent variable) that we want to predict.

- **Type:** Binary Classification (0 or 1)
- **Meaning:**
  - **0:** Pet was **NOT adopted** - remained in the shelter
  - **1:** Pet was **adopted** - successfully found a home

- **Distribution:**
  - **Not Adopted (0):** 1,348 pets (67.2%)
  - **Adopted (1):** 659 pets (32.8%)

- **Class Imbalance:** The dataset shows class imbalance with approximately **2:1 ratio** (more pets not adopted than adopted). This is important to consider when building machine learning models, as we may need to:
  - Use stratified sampling
  - Apply class weights
  - Use appropriate evaluation metrics (precision, recall, F1-score, AUC-ROC)
  - Consider oversampling/undersampling techniques (SMOTE, etc.)

**Why is this important?**
Understanding which factors contribute to successful adoptions can help shelters:
- Identify pets that may need extra attention or marketing
- Optimize adoption fees and vaccination programs
- Improve shelter conditions and reduce time-to-adoption
- Make data-driven decisions to increase overall adoption rates

### Variable Descriptions

| Variable | Type | Description | Values/Range |
|----------|------|-------------|--------------|
| **PetID** | Numerical (ID) | Unique identifier for each pet | 500 - 2506 |
| **PetType** | Categorical | Type of animal | Cat, Dog, Bird, Rabbit |
| **Breed** | Categorical | Specific breed of the pet | Golden Retriever, Persian, Parakeet, Rabbit, Labrador, Poodle |
| **AgeMonths** | Numerical | Age of the pet in months | 1 - 179 months |
| **Color** | Categorical | Primary color of the pet | White, Orange, Gray, Black, Brown |
| **Size** | Categorical (Ordinal) | Physical size category | Small, Medium, Large |
| **WeightKg** | Numerical | Weight of the pet in kilograms | 1.02 - 30.0 kg |
| **Vaccinated** | Binary | Vaccination status | 0 = Not vaccinated, 1 = Vaccinated |
| **HealthCondition** | Binary | Overall health status | 0 = Healthy, 1 = Has health issues |
| **TimeInShelterDays** | Numerical | Number of days spent in the shelter | 1 - 89 days |
| **AdoptionFee** | Numerical | Fee required to adopt the pet (in currency units) | 0 - 499 |
| **PreviousOwner** | Binary | Whether the pet had a previous owner | 0 = No previous owner, 1 = Had previous owner |
| **AdoptionLikelihood** | Binary (Target) | Whether the pet was adopted | 0 = Not adopted, 1 = Adopted |

## 3. Data Description

### Dataset Overview

This dataset contains information about **2,007 shelter pets** with **13 variables** (features). The primary goal is to predict the **likelihood of pet adoption** based on various characteristics such as pet type, physical attributes, health status, and shelter conditions.

**Task Type:** Binary Classification
- **Objective:** Predict whether a pet will be adopted (1) or not (0)
- **Use Case:** Help animal shelters identify factors that influence adoption success and optimize their strategies to improve adoption rates

## 4. Exploratory Data Analysis (EDA)

### 4.1 Target Distribution

In [ ]:
# Target variable distribution analysis
print("="*60)
print("TARGET VARIABLE: AdoptionLikelihood")
print("="*60)

# Class counts
print("\n1. Class Distribution:")
print("-" * 60)
target_counts = df['AdoptionLikelihood'].value_counts().sort_index()
target_pct = df['AdoptionLikelihood'].value_counts(normalize=True).sort_index() * 100

for label, count in target_counts.items():
    pct = target_pct[label]
    status = "Not Adopted" if label == 0 else "Adopted"
    print(f"   Class {label} ({status:12s}): {count:4d} pets ({pct:5.2f}%)")

# Class balance ratio
ratio = target_counts[0] / target_counts[1]
print(f"\n   Class Imbalance Ratio: {ratio:.2f}:1 (Not Adopted : Adopted)")

# Visualizations
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Bar plot
colors_bar = ['#e74c3c', '#2ecc71']
bars = axes[0].bar(target_counts.index, target_counts.values, color=colors_bar, 
                    edgecolor='black', linewidth=1.5, alpha=0.8)
axes[0].set_xlabel('Adoption Likelihood', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Number of Pets', fontsize=12, fontweight='bold')
axes[0].set_title('Target Distribution - Count', fontsize=14, fontweight='bold', pad=15)
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['Not Adopted (0)', 'Adopted (1)'])
axes[0].grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

# Pie chart
colors_pie = ['#e74c3c', '#2ecc71']
explode = (0.05, 0.05)
wedges, texts, autotexts = axes[1].pie(target_counts.values, 
                                         labels=['Not Adopted', 'Adopted'],
                                         autopct='%1.1f%%',
                                         colors=colors_pie,
                                         explode=explode,
                                         startangle=90,
                                         textprops={'fontsize': 11, 'fontweight': 'bold'})
axes[1].set_title('Target Distribution - Percentage', fontsize=14, fontweight='bold', pad=15)

# Percentage bar plot
bars2 = axes[2].barh(['Not Adopted', 'Adopted'], target_pct.values[::-1], 
                     color=colors_bar[::-1], edgecolor='black', linewidth=1.5, alpha=0.8)
axes[2].set_xlabel('Percentage (%)', fontsize=12, fontweight='bold')
axes[2].set_title('Class Balance', fontsize=14, fontweight='bold', pad=15)
axes[2].grid(axis='x', alpha=0.3, linestyle='--')

# Add percentage labels
for i, bar in enumerate(bars2):
    width = bar.get_width()
    axes[2].text(width, bar.get_y() + bar.get_height()/2.,
                f'{width:.1f}%',
                ha='left', va='center', fontsize=11, fontweight='bold', 
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("Key Observations:")
print("-" * 60)
print(f"✓ Dataset is IMBALANCED - {ratio:.2f}x more 'Not Adopted' than 'Adopted'")
print(f"✓ This imbalance should be addressed during modeling")
print("="*60)

### 4.2 Univariate Analysis

Analysis of individual features to understand their distributions and characteristics.

#### 4.2.1 Numerical Features Distribution

In [ ]:
# Analyze numerical features
numerical_features = ['AgeMonths', 'WeightKg', 'TimeInShelterDays', 'AdoptionFee']

print("="*70)
print("NUMERICAL FEATURES ANALYSIS")
print("="*70)

# Statistical summary
print("\nDescriptive Statistics:")
print("-"*70)
stats_df = df[numerical_features].describe().T
stats_df['skewness'] = df[numerical_features].skew()
stats_df['kurtosis'] = df[numerical_features].kurtosis()
print(stats_df.round(2))

# Visualizations - Histograms with KDE
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, col in enumerate(numerical_features):
    # Histogram with KDE
    axes[idx].hist(df[col], bins=30, color='steelblue', edgecolor='black', 
                   alpha=0.7, density=True, label='Histogram')
    
    # KDE overlay
    df[col].plot(kind='kde', ax=axes[idx], color='red', linewidth=2, 
                 label='KDE', secondary_y=False)
    
    # Statistics lines
    mean_val = df[col].mean()
    median_val = df[col].median()
    axes[idx].axvline(mean_val, color='darkgreen', linestyle='--', 
                      linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='orange', linestyle='--', 
                      linewidth=2, label=f'Median: {median_val:.2f}')
    
    axes[idx].set_title(f'{col} Distribution', fontsize=13, fontweight='bold', pad=10)
    axes[idx].set_xlabel(col, fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Density / Frequency', fontsize=11, fontweight='bold')
    axes[idx].legend(loc='upper right', fontsize=9)
    axes[idx].grid(alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("Key Observations:")
print("-"*70)
for col in numerical_features:
    skew = df[col].skew()
    if abs(skew) < 0.5:
        skew_type = "approximately symmetric"
    elif skew > 0:
        skew_type = "right-skewed (positive skew)"
    else:
        skew_type = "left-skewed (negative skew)"
    print(f"✓ {col}: {skew_type} (skewness = {skew:.2f})")
print("="*70)

#### 4.2.2 Numerical Features - Boxplots

In [ ]:
# Boxplots to identify outliers
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_features):
    # Create boxplot
    bp = axes[idx].boxplot([df[col]], vert=True, patch_artist=True,
                           labels=[col],
                           boxprops=dict(facecolor='lightblue', color='navy', linewidth=2),
                           whiskerprops=dict(color='navy', linewidth=1.5),
                           capprops=dict(color='navy', linewidth=1.5),
                           medianprops=dict(color='red', linewidth=2),
                           flierprops=dict(marker='o', markerfacecolor='red', 
                                          markersize=5, markeredgecolor='darkred'))
    
    # Add mean marker
    mean_val = df[col].mean()
    axes[idx].plot(1, mean_val, marker='D', markersize=10, 
                   color='green', label=f'Mean: {mean_val:.2f}')
    
    axes[idx].set_title(f'{col} - Boxplot', fontsize=13, fontweight='bold', pad=10)
    axes[idx].set_ylabel('Value', fontsize=11, fontweight='bold')
    axes[idx].grid(axis='y', alpha=0.3, linestyle='--')
    axes[idx].legend(loc='upper right')
    
    # Calculate outliers using IQR
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    
    # Add text with outlier info
    outlier_text = f'Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)'
    axes[idx].text(0.98, 0.02, outlier_text, transform=axes[idx].transAxes,
                   fontsize=10, verticalalignment='bottom', horizontalalignment='right',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

#### 4.2.3 Categorical Features Distribution

In [ ]:
# Analyze categorical features
categorical_features = ['PetType', 'Breed', 'Color', 'Size']

print("="*70)
print("CATEGORICAL FEATURES ANALYSIS")
print("="*70)

# Count and percentage for each feature
for feature in categorical_features:
    print(f"\n{feature}:")
    print("-"*70)
    counts = df[feature].value_counts()
    percentages = df[feature].value_counts(normalize=True) * 100
    summary = pd.DataFrame({
        'Count': counts,
        'Percentage': percentages
    })
    print(summary.round(2))

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
axes = axes.ravel()

colors_palette = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c']

for idx, col in enumerate(categorical_features):
    value_counts = df[col].value_counts()
    
    # Create bar chart
    bars = axes[idx].bar(range(len(value_counts)), value_counts.values, 
                         color=colors_palette[:len(value_counts)],
                         edgecolor='black', linewidth=1.5, alpha=0.8)
    
    axes[idx].set_xlabel(col, fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Count', fontsize=12, fontweight='bold')
    axes[idx].set_title(f'{col} Distribution', fontsize=14, fontweight='bold', pad=10)
    axes[idx].set_xticks(range(len(value_counts)))
    axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
    axes[idx].grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                      f'{int(height)}',
                      ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*70)

#### 4.2.4 Binary Features Distribution

In [ ]:
# Analyze binary features
binary_features = ['Vaccinated', 'HealthCondition', 'PreviousOwner']
binary_labels = {
    'Vaccinated': {0: 'Not Vaccinated', 1: 'Vaccinated'},
    'HealthCondition': {0: 'Healthy', 1: 'Has Issues'},
    'PreviousOwner': {0: 'No Previous Owner', 1: 'Had Previous Owner'}
}

print("="*70)
print("BINARY FEATURES ANALYSIS")
print("="*70)

# Statistics
for feature in binary_features:
    print(f"\n{feature}:")
    print("-"*70)
    counts = df[feature].value_counts().sort_index()
    percentages = df[feature].value_counts(normalize=True).sort_index() * 100
    
    for val in [0, 1]:
        label = binary_labels[feature][val]
        print(f"   {val} ({label:20s}): {counts[val]:4d} ({percentages[val]:5.2f}%)")

# Visualizations
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

colors = ['#e74c3c', '#2ecc71']

for idx, feature in enumerate(binary_features):
    counts = df[feature].value_counts().sort_index()
    labels = [binary_labels[feature][0], binary_labels[feature][1]]
    
    # Bar chart
    bars = axes[idx].bar(labels, counts.values, color=colors,
                         edgecolor='black', linewidth=1.5, alpha=0.8)
    
    axes[idx].set_xlabel(feature, fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Count', fontsize=12, fontweight='bold')
    axes[idx].set_title(f'{feature} Distribution', fontsize=14, fontweight='bold', pad=10)
    axes[idx].tick_params(axis='x', rotation=15)
    axes[idx].grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add percentage labels
    for bar, count in zip(bars, counts.values):
        height = bar.get_height()
        pct = (count / len(df)) * 100
        axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                      f'{int(count)}\n({pct:.1f}%)',
                      ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*70)

### 4.3 Bivariate Analysis

Analyzing relationships between features and the target variable (AdoptionLikelihood).

#### 4.3.1 Adoption Rate by Categorical Features

In [ ]:
# Boxplots for numerical features grouped by target
numerical_features = ['AgeMonths', 'WeightKg', 'TimeInShelterDays', 'AdoptionFee']

print("="*70)
print("NUMERICAL FEATURES BY ADOPTION OUTCOME")
print("="*70)

# Statistical comparison
for feature in numerical_features:
    print(f"\n{feature}:")
    print("-"*70)
    stats_by_target = df.groupby('AdoptionLikelihood')[feature].describe()
    print(stats_by_target.round(2))

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

colors = {0: '#e74c3c', 1: '#2ecc71'}

for idx, col in enumerate(numerical_features):
    # Prepare data
    not_adopted = df[df['AdoptionLikelihood'] == 0][col]
    adopted = df[df['AdoptionLikelihood'] == 1][col]
    
    # Create boxplot
    bp = axes[idx].boxplot([not_adopted, adopted], 
                           labels=['Not Adopted (0)', 'Adopted (1)'],
                           patch_artist=True,
                           boxprops=dict(linewidth=2),
                           whiskerprops=dict(linewidth=1.5),
                           capprops=dict(linewidth=1.5),
                           medianprops=dict(color='darkblue', linewidth=2),
                           flierprops=dict(marker='o', markersize=4, alpha=0.5))
    
    # Color the boxes
    for patch, adoption_status in zip(bp['boxes'], [0, 1]):
        patch.set_facecolor(colors[adoption_status])
        patch.set_alpha(0.7)
    
    axes[idx].set_title(f'{col} by Adoption Outcome', fontsize=13, fontweight='bold', pad=10)
    axes[idx].set_ylabel(col, fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Adoption Outcome', fontsize=11, fontweight='bold')
    axes[idx].grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add mean markers
    mean_not_adopted = not_adopted.mean()
    mean_adopted = adopted.mean()
    axes[idx].plot(1, mean_not_adopted, 'D', markersize=10, color='darkred', 
                   label=f'Mean Not Adopted: {mean_not_adopted:.1f}')
    axes[idx].plot(2, mean_adopted, 'D', markersize=10, color='darkgreen',
                   label=f'Mean Adopted: {mean_adopted:.1f}')
    axes[idx].legend(loc='best', fontsize=9)

plt.tight_layout()
plt.show()

print("\n" + "="*70)

In [ ]:
# Missing values analysis
print("="*70)
print("MISSING VALUES ANALYSIS")
print("="*70)

missing_counts = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Percentage (%)': missing_pct
}).sort_values('Missing Count', ascending=False)

print("\nMissing Values Summary:")
print("-"*70)
if missing_df['Missing Count'].sum() == 0:
    print("✓ NO MISSING VALUES FOUND - Dataset is complete!")
else:
    print(missing_df[missing_df['Missing Count'] > 0])

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Missing values bar chart
if missing_df['Missing Count'].sum() > 0:
    missing_features = missing_df[missing_df['Missing Count'] > 0]
    axes[0].bar(range(len(missing_features)), missing_features['Percentage (%)'].values,
                color='#e74c3c', edgecolor='black', linewidth=1.5, alpha=0.8)
    axes[0].set_xticks(range(len(missing_features)))
    axes[0].set_xticklabels(missing_features.index, rotation=45, ha='right')
    axes[0].set_ylabel('Missing Percentage (%)', fontsize=12, fontweight='bold')
    axes[0].set_title('Missing Values by Feature', fontsize=14, fontweight='bold', pad=10)
    axes[0].grid(axis='y', alpha=0.3, linestyle='--')
else:
    axes[0].text(0.5, 0.5, 'No Missing Values\n✓ Complete Dataset', 
                ha='center', va='center', fontsize=16, fontweight='bold',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
    axes[0].axis('off')

# Heatmap showing completeness
complete_matrix = df.notnull().astype(int)
sns.heatmap(complete_matrix.T, cmap='RdYlGn', cbar_kws={'label': 'Complete (1) / Missing (0)'},
            yticklabels=df.columns, ax=axes[1], xticklabels=False)
axes[1].set_title('Data Completeness Heatmap', fontsize=14, fontweight='bold', pad=10)
axes[1].set_xlabel('Samples', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Outliers analysis
print("\n" + "="*70)
print("OUTLIERS ANALYSIS (IQR Method)")
print("="*70)

numerical_features = ['AgeMonths', 'WeightKg', 'TimeInShelterDays', 'AdoptionFee']

outlier_summary = []

for feature in numerical_features:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    outlier_count = len(outliers)
    outlier_pct = (outlier_count / len(df)) * 100
    
    outlier_summary.append({
        'Feature': feature,
        'Q1': Q1,
        'Q3': Q3,
        'IQR': IQR,
        'Lower Bound': lower_bound,
        'Upper Bound': upper_bound,
        'Outlier Count': outlier_count,
        'Outlier %': outlier_pct
    })

outlier_df = pd.DataFrame(outlier_summary)

print("\nOutlier Detection Summary:")
print("-"*70)
print(outlier_df.round(2))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, feature in enumerate(numerical_features):
    # Calculate outliers
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Separate outliers and normal values
    normal_mask = (df[feature] >= lower_bound) & (df[feature] <= upper_bound)
    outlier_mask = ~normal_mask
    
    # Scatter plot
    axes[idx].scatter(df[normal_mask].index, df[normal_mask][feature],
                     alpha=0.5, s=20, color='steelblue', label='Normal')
    axes[idx].scatter(df[outlier_mask].index, df[outlier_mask][feature],
                     alpha=0.7, s=30, color='red', marker='x', label='Outliers')
    
    # Add boundary lines
    axes[idx].axhline(y=lower_bound, color='orange', linestyle='--', 
                     linewidth=2, label=f'Lower Bound: {lower_bound:.1f}')
    axes[idx].axhline(y=upper_bound, color='green', linestyle='--', 
                     linewidth=2, label=f'Upper Bound: {upper_bound:.1f}')
    
    axes[idx].set_title(f'{feature} - Outlier Detection', fontsize=13, fontweight='bold', pad=10)
    axes[idx].set_xlabel('Sample Index', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel(feature, fontsize=11, fontweight='bold')
    axes[idx].legend(loc='best', fontsize=9)
    axes[idx].grid(alpha=0.3, linestyle='--')
    
    # Add text box with outlier count
    outlier_count = outlier_mask.sum()
    outlier_pct = (outlier_count / len(df)) * 100
    textstr = f'Outliers: {outlier_count}\n({outlier_pct:.1f}%)'
    axes[idx].text(0.02, 0.98, textstr, transform=axes[idx].transAxes,
                  fontsize=10, verticalalignment='top',
                  bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("Summary:")
print("-"*70)
total_outliers = outlier_df['Outlier Count'].sum()
print(f"✓ Total outliers across all numerical features: {total_outliers}")
print(f"✓ Average outlier percentage: {outlier_df['Outlier %'].mean():.2f}%")
if total_outliers == 0:
    print("✓ No outliers detected - data is well-distributed!")
else:
    print("✓ Outliers detected but kept for analysis - consider handling in preprocessing")
print("="*70)

In [ ]:
# Key Insights Summary
print("="*80)
print(" "*25 + "KEY INSIGHTS FROM EDA")
print("="*80)

# 1. Dataset Overview
print("\n1. DATASET OVERVIEW")
print("-"*80)
print(f"   • Total samples: {len(df):,}")
print(f"   • Total features: {df.shape[1]}")
print(f"   • Numerical features: {len(df.select_dtypes(include=[np.number]).columns)}")
print(f"   • Categorical features: {len(df.select_dtypes(include=['object']).columns)}")
print(f"   • Missing values: {df.isnull().sum().sum()} (Dataset is complete!)")

# 2. Target Variable
print("\n2. TARGET VARIABLE (AdoptionLikelihood)")
print("-"*80)
adopted = df['AdoptionLikelihood'].sum()
not_adopted = len(df) - adopted
adoption_rate = df['AdoptionLikelihood'].mean()
imbalance_ratio = not_adopted / adopted

print(f"   • Adoption rate: {adoption_rate:.1%}")
print(f"   • Adopted: {adopted} pets ({adopted/len(df)*100:.1f}%)")
print(f"   • Not adopted: {not_adopted} pets ({not_adopted/len(df)*100:.1f}%)")
print(f"   • Class imbalance ratio: {imbalance_ratio:.2f}:1 (Not Adopted : Adopted)")
print(f"   ⚠ ACTION NEEDED: Address class imbalance in modeling (use SMOTE, class weights, etc.)")

# 3. Feature Analysis
print("\n3. FEATURE CHARACTERISTICS")
print("-"*80)

# Numerical features
numerical_features = ['AgeMonths', 'WeightKg', 'TimeInShelterDays', 'AdoptionFee']
print("   Numerical Features:")
for feature in numerical_features:
    mean_val = df[feature].mean()
    median_val = df[feature].median()
    std_val = df[feature].std()
    skew_val = df[feature].skew()
    print(f"   • {feature:20s}: Mean={mean_val:6.1f}, Median={median_val:6.1f}, Std={std_val:6.1f}, Skew={skew_val:5.2f}")

# Categorical features
print("\n   Categorical Features:")
categorical_features = ['PetType', 'Breed', 'Color', 'Size']
for feature in categorical_features:
    n_unique = df[feature].nunique()
    most_common = df[feature].value_counts().index[0]
    most_common_pct = df[feature].value_counts(normalize=True).iloc[0] * 100
    print(f"   • {feature:20s}: {n_unique} unique values (Most common: {most_common}, {most_common_pct:.1f}%)")

# 4. Correlation with Target
print("\n4. CORRELATION WITH TARGET")
print("-"*80)
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
correlation_matrix = df[numerical_cols].corr()
target_corr = correlation_matrix['AdoptionLikelihood'].drop('AdoptionLikelihood').sort_values(ascending=False)

print("   Top features correlated with adoption:")
for idx, (feature, corr_value) in enumerate(target_corr.head(5).items(), 1):
    direction = "positively" if corr_value > 0 else "negatively"
    print(f"   {idx}. {feature:20s}: {corr_value:+.3f} ({direction} correlated)")

# 5. Adoption Rates by Features
print("\n5. ADOPTION RATES BY KEY FEATURES")
print("-"*80)

# Best adoption rates
print("   Features with HIGHEST adoption rates:")
features_to_check = ['PetType', 'Size', 'Vaccinated']
for feature in features_to_check:
    best_category = df.groupby(feature)['AdoptionLikelihood'].mean().idxmax()
    best_rate = df.groupby(feature)['AdoptionLikelihood'].mean().max()
    print(f"   • {feature:20s}: {best_category} ({best_rate:.1%} adoption rate)")

print("\n   Features with LOWEST adoption rates:")
for feature in features_to_check:
    worst_category = df.groupby(feature)['AdoptionLikelihood'].mean().idxmin()
    worst_rate = df.groupby(feature)['AdoptionLikelihood'].mean().min()
    print(f"   • {feature:20s}: {worst_category} ({worst_rate:.1%} adoption rate)")

# 6. Data Quality
print("\n6. DATA QUALITY")
print("-"*80)
print(f"   • Missing values: ✓ None (100% complete)")
print(f"   • Duplicate rows: ✓ None")

# Check outliers
outlier_counts = []
for feature in numerical_features:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    outlier_counts.append(len(outliers))

total_outliers = sum(outlier_counts)
if total_outliers == 0:
    print(f"   • Outliers: ✓ None detected")
else:
    print(f"   • Outliers: {total_outliers} detected across numerical features (kept for analysis)")

# 7. Key Recommendations
print("\n7. KEY RECOMMENDATIONS FOR MODELING")
print("-"*80)
print("   1. ⚠ Address class imbalance (67% not adopted vs 33% adopted)")
print("      → Use techniques like SMOTE, class weights, or stratified sampling")
print("   ")
print("   2. ✓ No missing values - dataset is ready for modeling")
print("   ")
print("   3. 📊 Focus on important features:")

# Get top features
top_features_list = target_corr.head(3).index.tolist()
for idx, feature in enumerate(top_features_list, 1):
    print(f"      → Feature {idx}: {feature} (correlation: {target_corr[feature]:+.3f})")

print("   ")
print("   4. 🔍 Consider feature engineering:")
print("      → Age categories (Young/Adult/Senior)")
print("      → Combined health scores (Vaccination + Health Condition)")
print("      → Time-based features (days in shelter categories)")
print("   ")
print("   5. 📈 Evaluation metrics to use:")
print("      → Precision, Recall, F1-Score (due to imbalanced classes)")
print("      → ROC-AUC curve")
print("      → Confusion matrix analysis")

print("\n" + "="*80)
print(" "*20 + "END OF EXPLORATORY DATA ANALYSIS")
print("="*80)

### 4.6 Key Insights Summary

This section summarizes the most important findings from our exploratory data analysis.

### 4.5 Missing Values and Outliers Analysis

In [ ]:
# Correlation matrix for all numerical features
print("="*70)
print("CORRELATION ANALYSIS")
print("="*70)

# Select numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Calculate correlation matrix
correlation_matrix = df[numerical_cols].corr()

print("\nCorrelation with Target Variable (AdoptionLikelihood):")
print("-"*70)
target_corr = correlation_matrix['AdoptionLikelihood'].sort_values(ascending=False)
for feature, corr_value in target_corr.items():
    if feature != 'AdoptionLikelihood':
        if abs(corr_value) > 0.3:
            strength = "Strong"
        elif abs(corr_value) > 0.1:
            strength = "Moderate"
        else:
            strength = "Weak"
        print(f"{feature:25s}: {corr_value:6.3f} ({strength})")

# Visualization - Heatmap
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Full correlation matrix
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            ax=axes[0], vmin=-1, vmax=1)
axes[0].set_title('Correlation Matrix - All Numerical Features', 
                  fontsize=14, fontweight='bold', pad=15)

# Correlation with target only
target_corr_df = target_corr.drop('AdoptionLikelihood').to_frame()
target_corr_df.columns = ['Correlation']
sns.heatmap(target_corr_df, annot=True, fmt='.3f', cmap='RdYlGn', 
            center=0, linewidths=1, cbar_kws={"shrink": 0.8},
            ax=axes[1], vmin=-1, vmax=1)
axes[1].set_title('Correlation with Target (AdoptionLikelihood)', 
                  fontsize=14, fontweight='bold', pad=15)

plt.tight_layout()
plt.show()

# Additional: Scatter plots for top correlated features
top_features = target_corr.drop('AdoptionLikelihood').head(4).index.tolist()

if len(top_features) >= 2:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx, feature in enumerate(top_features[:4]):
        # Scatter plot
        for adoption_status in [0, 1]:
            mask = df['AdoptionLikelihood'] == adoption_status
            label = 'Adopted' if adoption_status == 1 else 'Not Adopted'
            color = '#2ecc71' if adoption_status == 1 else '#e74c3c'
            axes[idx].scatter(df[mask][feature], df[mask]['AdoptionLikelihood'],
                            alpha=0.5, s=30, label=label, color=color)
        
        axes[idx].set_xlabel(feature, fontsize=11, fontweight='bold')
        axes[idx].set_ylabel('AdoptionLikelihood', fontsize=11, fontweight='bold')
        axes[idx].set_title(f'{feature} vs AdoptionLikelihood (r={target_corr[feature]:.3f})',
                           fontsize=12, fontweight='bold', pad=10)
        axes[idx].legend()
        axes[idx].grid(alpha=0.3, linestyle='--')
    
    plt.tight_layout()
    plt.show()

print("\n" + "="*70)

### 4.4 Correlation Analysis

#### 4.3.2 Numerical Features by Target (Boxplots)

In [ ]:
# Adoption rate by categorical features
all_features = ['PetType', 'Breed', 'Color', 'Size', 'Vaccinated', 'HealthCondition', 'PreviousOwner']

print("="*70)
print("ADOPTION RATE BY CATEGORICAL FEATURES")
print("="*70)

# Calculate adoption rates
overall_adoption_rate = df['AdoptionLikelihood'].mean()
print(f"\nOverall Adoption Rate: {overall_adoption_rate:.2%}")
print("-"*70)

# Statistics for each feature
for feature in all_features:
    print(f"\n{feature}:")
    adoption_by_feature = df.groupby(feature)['AdoptionLikelihood'].agg(['mean', 'count'])
    adoption_by_feature.columns = ['Adoption Rate', 'Count']
    adoption_by_feature['Adoption Rate'] = adoption_by_feature['Adoption Rate'] * 100
    print(adoption_by_feature.round(2))

# Visualizations
fig, axes = plt.subplots(3, 3, figsize=(20, 16))
axes = axes.ravel()

for idx, feature in enumerate(all_features):
    if idx < len(axes):
        adoption_by_feature = df.groupby(feature)['AdoptionLikelihood'].mean().sort_values(ascending=False)
        
        # Create bar chart
        bars = axes[idx].bar(range(len(adoption_by_feature)), 
                            adoption_by_feature.values * 100,
                            color='#2ecc71', edgecolor='black', linewidth=1.5, alpha=0.8)
        
        axes[idx].set_xlabel(feature, fontsize=12, fontweight='bold')
        axes[idx].set_ylabel('Adoption Rate (%)', fontsize=12, fontweight='bold')
        axes[idx].set_title(f'Adoption Rate by {feature}', fontsize=13, fontweight='bold', pad=10)
        axes[idx].set_xticks(range(len(adoption_by_feature)))
        axes[idx].set_xticklabels(adoption_by_feature.index, rotation=45, ha='right')
        axes[idx].grid(axis='y', alpha=0.3, linestyle='--')
        
        # Add horizontal line for overall mean
        axes[idx].axhline(y=overall_adoption_rate * 100, color='red', linestyle='--',
                         linewidth=2, label=f'Overall: {overall_adoption_rate*100:.1f}%')
        
        # Add percentage labels on bars
        for bar in bars:
            height = bar.get_height()
            axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                          f'{height:.1f}%',
                          ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        axes[idx].legend(loc='upper right')

# Remove extra subplots
for idx in range(len(all_features), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

print("\n" + "="*70)

## 6. Data Cleaning and Feature Engineering

### 6.1 Handle Missing Values and Duplicates

In [28]:
# Create a copy for cleaning
df_clean = df.copy()

print("Initial dataset shape:", df_clean.shape)

# Handle duplicates
duplicates = df_clean.duplicated().sum()
if duplicates > 0:
    print(f"Removing {duplicates} duplicate rows...")
    df_clean = df_clean.drop_duplicates()
else:
    print("No duplicates found.")

# Handle missing values
missing = df_clean.isnull().sum().sum()
if missing > 0:
    print(f"\nHandling {missing} missing values...")
    # Strategy: fill numerical with median, categorical with mode
    for col in df_clean.columns:
        if df_clean[col].isnull().sum() > 0:
            if df_clean[col].dtype == 'object':
                df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
            else:
                df_clean[col].fillna(df_clean[col].median(), inplace=True)
else:
    print("No missing values found.")

print("\nCleaned dataset shape:", df_clean.shape)
print("Missing values after cleaning:", df_clean.isnull().sum().sum())

Initial dataset shape: (2007, 13)
No duplicates found.
No missing values found.

Cleaned dataset shape: (2007, 13)
Missing values after cleaning: 0


### 6.2 Handle Outliers

In [29]:
# Detect and handle outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

numerical_features = ['AgeMonths', 'WeightKg', 'TimeInShelterDays', 'AdoptionFee']

print("Outlier Detection Summary:")
print("="*50)
for col in numerical_features:
    outliers, lower, upper = detect_outliers_iqr(df_clean, col)
    print(f"\n{col}:")
    print(f"  Outliers found: {len(outliers)}")
    print(f"  Lower bound: {lower:.2f}, Upper bound: {upper:.2f}")
    
# For now, we'll keep outliers but note them
# In production, you might want to cap or remove them based on domain knowledge
print("\n\nNote: Outliers are kept in the dataset for now.")
print("Consider capping or removing them based on domain expertise.")

Outlier Detection Summary:

AgeMonths:
  Outliers found: 0
  Lower bound: -87.00, Upper bound: 273.00

WeightKg:
  Outliers found: 0
  Lower bound: -12.28, Upper bound: 43.75

TimeInShelterDays:
  Outliers found: 0
  Lower bound: -46.50, Upper bound: 133.50

AdoptionFee:
  Outliers found: 0
  Lower bound: -245.00, Upper bound: 747.00


Note: Outliers are kept in the dataset for now.
Consider capping or removing them based on domain expertise.


### 6.3 Feature Engineering

In [30]:
# Create new features
# Check if df_clean exists, if not create it from df
if 'df_clean' not in dir():
    df_clean = df.copy()
    print("Note: df_clean created from df (cleaning steps not run)")

df_engineered = df_clean.copy()

# 1. Age categories
def categorize_age(age):
    if age < 12:
        return 'Young'
    elif age < 60:
        return 'Adult'
    else:
        return 'Senior'

df_engineered['AgeCategory'] = df_engineered['AgeMonths'].apply(categorize_age)

# 2. Weight categories based on size
df_engineered['WeightPerSize'] = df_engineered['WeightKg'] / df_engineered['Size'].map({'Small': 1, 'Medium': 2, 'Large': 3})

# 3. Shelter time categories
def categorize_shelter_time(days):
    if days < 30:
        return 'Short'
    elif days < 90:
        return 'Medium'
    else:
        return 'Long'

df_engineered['ShelterTimeCategory'] = df_engineered['TimeInShelterDays'].apply(categorize_shelter_time)

# 4. Fee range categories
df_engineered['FeeRange'] = pd.cut(df_engineered['AdoptionFee'], bins=3, labels=['Low', 'Medium', 'High'])

# 5. Health score (combination of vaccination and health condition)
df_engineered['HealthScore'] = df_engineered['Vaccinated'] + df_engineered['HealthCondition']

# 6. Age in years
df_engineered['AgeYears'] = df_engineered['AgeMonths'] / 12

print("New Features Created:")
print("="*50)
print("1. AgeCategory - Categorical age groups")
print("2. WeightPerSize - Weight normalized by size")
print("3. ShelterTimeCategory - Time in shelter categories")
print("4. FeeRange - Adoption fee ranges")
print("5. HealthScore - Combined health indicator")
print("6. AgeYears - Age in years")

print(f"\nNew dataset shape: {df_engineered.shape}")
print(f"\nSample of engineered features:")
df_engineered[['AgeMonths', 'AgeCategory', 'AgeYears', 'WeightKg', 'WeightPerSize', 
               'TimeInShelterDays', 'ShelterTimeCategory', 'HealthScore']].head()

New Features Created:
1. AgeCategory - Categorical age groups
2. WeightPerSize - Weight normalized by size
3. ShelterTimeCategory - Time in shelter categories
4. FeeRange - Adoption fee ranges
5. HealthScore - Combined health indicator
6. AgeYears - Age in years

New dataset shape: (2007, 19)

Sample of engineered features:


,AgeMonths,AgeCategory,AgeYears,WeightKg,WeightPerSize,TimeInShelterDays,ShelterTimeCategory,HealthScore
0,131,Senior,10.916667,5.039768,1.679923,27,Short,1
1,73,Senior,6.083333,16.086727,5.362242,8,Short,0
2,136,Senior,11.333333,2.076286,1.038143,85,Medium,0
3,97,Senior,8.083333,3.339423,3.339423,61,Medium,0
4,123,Senior,10.250000,20.498100,6.832700,28,Short,0


### 6.4 Encode Categorical Variables

In [31]:
# Encode categorical variables
# Check if df_engineered exists, if not create it from df_clean or df
if 'df_engineered' not in dir():
    if 'df_clean' in dir():
        print("Warning: df_engineered not found. Run the Feature Engineering cell first!")
        print("Creating df_engineered from df_clean without feature engineering...")
        df_engineered = df_clean.copy()
    else:
        print("Warning: Both df_engineered and df_clean not found. Run previous cells first!")
        print("Creating from df...")
        df_engineered = df.copy()

df_encoded = df_engineered.copy()

# One-hot encoding for nominal categorical variables
nominal_features = ['PetType', 'Breed', 'Color', 'Size']

# Add engineered features if they exist
if 'AgeCategory' in df_encoded.columns:
    nominal_features.extend(['AgeCategory', 'ShelterTimeCategory', 'FeeRange'])

# Apply one-hot encoding
df_encoded = pd.get_dummies(df_encoded, columns=nominal_features, prefix=nominal_features, drop_first=True)

print("Encoding Summary:")
print("="*50)
print(f"Original shape: {df_engineered.shape}")
print(f"After encoding shape: {df_encoded.shape}")
print(f"\nNew columns added: {df_encoded.shape[1] - df_engineered.shape[1]}")

print("\nFirst few columns after encoding:")
print(df_encoded.columns.tolist()[:20])

Encoding Summary:
Original shape: (2007, 19)
After encoding shape: (2007, 32)

New columns added: 13

First few columns after encoding:
['PetID', 'AgeMonths', 'WeightKg', 'Vaccinated', 'HealthCondition', 'TimeInShelterDays', 'AdoptionFee', 'PreviousOwner', 'AdoptionLikelihood', 'WeightPerSize', 'HealthScore', 'AgeYears', 'PetType_Cat', 'PetType_Dog', 'PetType_Rabbit', 'Breed_Labrador', 'Breed_Parakeet', 'Breed_Persian', 'Breed_Poodle', 'Breed_Rabbit']


### 6.5 Prepare Final Dataset

In [32]:
# Prepare final dataset for modeling
df_final = df_encoded.copy()

# Drop PetID as it's just an identifier
if 'PetID' in df_final.columns:
    df_final = df_final.drop('PetID', axis=1)

# Separate features and target
X = df_final.drop('AdoptionLikelihood', axis=1)
y = df_final['AdoptionLikelihood']

print("Final Dataset Summary:")
print("="*50)
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nNumber of features: {X.shape[1]}")
print(f"\nTarget distribution:")
print(y.value_counts())
print(f"\nClass balance: {y.value_counts(normalize=True) * 100}")

print("\n" + "="*50)
print("Data preparation complete!")
print("Dataset is ready for model training.")

Final Dataset Summary:
Features shape: (2007, 30)
Target shape: (2007,)

Number of features: 30

Target distribution:
AdoptionLikelihood
0    1348
1     659
Name: count, dtype: int64

Class balance: AdoptionLikelihood
0    67.164923
1    32.835077
Name: proportion, dtype: float64

Data preparation complete!
Dataset is ready for model training.


### 6.6 Save Processed Data

In [33]:
# Save processed datasets for future use
# Uncomment the lines below to save the processed data

# df_clean.to_csv('pet_adoption_cleaned.csv', index=False)
# df_engineered.to_csv('pet_adoption_engineered.csv', index=False)
# df_final.to_csv('pet_adoption_final.csv', index=False)

print("Processed datasets ready to be saved:")
print("1. df_clean - Cleaned data (duplicates and missing values handled)")
print("2. df_engineered - With engineered features")
print("3. df_final - Fully processed and encoded, ready for modeling")
print("\nUncomment the save commands above to export the data.")

Processed datasets ready to be saved:
1. df_clean - Cleaned data (duplicates and missing values handled)
2. df_engineered - With engineered features
3. df_final - Fully processed and encoded, ready for modeling

Uncomment the save commands above to export the data.
